In [1]:
#import libraries
import pandas as pd
import seaborn as sns
import numpy as np
import plotly
import plotly.express as px
import warnings
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio

warnings.filterwarnings('ignore')
sns.set(font_scale=1.5)
plt.rcParams['figure.figsize'] = (12, 8)

In [2]:
#install linearmodels
!pip install linearmodels

from linearmodels import PanelOLS
from linearmodels import RandomEffects
import statsmodels.formula.api as smf
from linearmodels.panel import compare

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 1.7 MB/s eta 0:00:00


In [10]:
#read in the dataset
df = pd.read_csv('Brazilnewlocationdata.csv')
df.head()

,measure,location,sex,age,cause,metric,year,val,upper,lower
0,Prevalence,Mato Grosso do Sul,Both,All ages,Diabetes mellitus type 2,Number,1990,48971.383010,54174.689470,43968.234840
1,Prevalence,Mato Grosso do Sul,Both,All ages,Diabetes mellitus type 2,Percent,1990,0.028682,0.031747,0.025762
2,Prevalence,Mato Grosso do Sul,Both,All ages,Diabetes mellitus type 2,Rate,1990,2730.838086,3020.995043,2451.842748
3,Prevalence,Mato Grosso do Sul,Both,All ages,Diabetes mellitus type 2,Number,1991,50680.883560,56118.188880,45557.249960
4,Prevalence,Mato Grosso do Sul,Both,All ages,Diabetes mellitus type 2,Percent,1991,0.029124,0.032218,0.026150


In [11]:
#refine the dataframe
df['val_percent'] = df['val']*100
df=df[df['measure']=='Prevalence']
df=df[df['metric']=='Percent']
df.head()

,measure,location,sex,age,cause,metric,year,val,upper,lower,val_percent
1,Prevalence,Mato Grosso do Sul,Both,All ages,Diabetes mellitus type 2,Percent,1990,0.028682,0.031747,0.025762,2.868202
4,Prevalence,Mato Grosso do Sul,Both,All ages,Diabetes mellitus type 2,Percent,1991,0.029124,0.032218,0.026150,2.912357
7,Prevalence,Mato Grosso do Sul,Both,All ages,Diabetes mellitus type 2,Percent,1992,0.029636,0.032689,0.026602,2.963580
10,Prevalence,Mato Grosso do Sul,Both,All ages,Diabetes mellitus type 2,Percent,1993,0.030186,0.033282,0.027030,3.018603
13,Prevalence,Mato Grosso do Sul,Both,All ages,Diabetes mellitus type 2,Percent,1994,0.030768,0.034174,0.027368,3.076790


In [12]:
#create a new dataframe only including Amapá and Pará
did=df[df['location'].isin(['Amapá', 'Pará'])]
did.head()

,measure,location,sex,age,cause,metric,year,val,upper,lower,val_percent
1171,Prevalence,Amapá,Both,All ages,Diabetes mellitus type 2,Percent,1990,0.017306,0.019288,0.015551,1.730590
1174,Prevalence,Amapá,Both,All ages,Diabetes mellitus type 2,Percent,1991,0.017364,0.019318,0.015628,1.736433
1177,Prevalence,Amapá,Both,All ages,Diabetes mellitus type 2,Percent,1992,0.017426,0.019272,0.015723,1.742555
1180,Prevalence,Amapá,Both,All ages,Diabetes mellitus type 2,Percent,1993,0.017492,0.019376,0.015774,1.749155
1183,Prevalence,Amapá,Both,All ages,Diabetes mellitus type 2,Percent,1994,0.017568,0.019508,0.015815,1.756828


when plotting the graph, i wanted to add two vertical lines to represent the start and end of the barge implementation.

here is the link to the plot.ly documentation used for this: https://plotly.com/python/horizontal-vertical-shapes/

In [6]:
#plot a line graph of diabetes prevalence in amapá and pará over time.
fig = px.line(did, x='year', y='val_percent', color='location',
              labels={
                     "year": "Year",
                     "val_percent": "Diabetes Prevalence %"
                 }, title="Diabetes Prevalence (percentage) in Amapá and Pará")

fig.add_vline(x=2010, line_width=3, line_dash="dash", line_color="white", annotation_text=" Barge implementation",
              annotation_position="top right")
fig.add_vline(x=2017, line_width=3, line_dash="dash", line_color="white", annotation_text=" Barge scheme ends",
              annotation_position="bottom right")
fig.update_layout(template='plotly_dark',plot_bgcolor='black', paper_bgcolor='black')
fig.show()

when attempting to export the interactive plot.ly graph, i came across an issue. the size of the html file is ENORMOUS and basically crashed the entire wix website when i tried to embed it in. this is because the original html file includes the plotlyjs source code, therefore i opted out in including the plotlyjs source code, given that we are presenting the graph on a website, which needs internet connection anyway.

here is a link to the stackoverflow page i used in order to address the issue.
https://stackoverflow.com/questions/72498679/plotly-how-to-avoid-huge-html-file-size

In [7]:
#export figure (including html file compression).
f = open('filename_here.html', "w")
f.close()
with open('filename_here.html', 'a') as f:
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
f.close()

In [17]:
#creating variables
did['post']=np.where(did['year']>=2010,1,0)
did['treatment']=np.where(did['location']=='Pará',1,0)
did['post_treatment']=did['post']*did['treatment']

In [18]:
#performing a difference-in-difference analysis
did_model = ols('val_percent ~ post + treatment + post_treatment', did).fit()
print(did_model.summary())

                            OLS Regression Results                            
Dep. Variable:            val_percent   R-squared:                       0.792
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                     70.93
Date:                Sat, 06 Jan 2024   Prob (F-statistic):           4.56e-19
Time:                        14:11:16   Log-Likelihood:                -25.573
No. Observations:                  60   AIC:                             59.15
Df Residuals:                      56   BIC:                             67.52
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          2.0018      0.086     23.

In [19]:
# Silly Nyree...

  When analysing the results of our difference in difference regression, just as we must satisfy assumptions such as multicollinearity, we must satisfy two key assumptions:
  
  Firstly, to satisfy that there are no simultaneous treatments, we mustmake sure that, taking Amapá as the control group, there were no key events or changed in unconsidered variables in 2010 that may have had an effect on diabetes prevalence. To account for this, we should also observe a line graph for another variable in Amapá, that also could have influenced this change. If we do not do this, we cannot statistically confirm that the treatment group (Barge in Pará) was the reason for the change.

  Secondly, when we take a control group, it must be equivalent to the treatment group when then the treatment is not applied. This means that Amapá and Pará must have had very similar trend in diabetes prevalence before the treatment (Barge) is implemented. Here, we should also make a plot of prevalence over time, to make sure we observe this. Without this, we cannot argue that the treatment was the cause for the change in diabetes prevalence in Amapá.

  What is effective about this regression is rather than just showing a correlation between results, this regression is causal, meaning it gives sufficient statistical proof of causality, meaning that this test could prove that the implementation of Barge in Pará cause the change in diabetes prevalence in Amapá. Now, let us analyse the results.

  


Here, our null hypothesis is that there is no correlation between any of these variables, and any patterns or trends observed are likely due to random chance. We are going to perform a hypotheseis test with a 1% significance level, to test if we can accept this null hypothesis.

The 'post' coefficient tells us that in general, over time, after the introduction of Barge, there was an increase of ~1.3% of diabetes prevalence in Amapá and Pará. This is statistically significant at the 1% significance level, and so we can reject the null hypothesis, suggesting that this is not by random chance.

The 'treatment' coefficient tells us that, regardless of any time period, the average change in diabetes prevalence after the introduction of Barge was in increase of ~0.49%, which is also statistically significant

What is most important to us is the 'post-treatment' coefficient, which determines the causal effect of the treatment onto the control group, which shows that the introduction of Barge in Pará led to a ~0.24% increase in diabetes prevalence in Amapá. However, this is not statistically significant, as its p-value is not within our assigned critical region, and so this may be due to random chance, and we cannot determine if there is a causal relationship. However, this may be due to the range of years we have chosen, as we have chosen a premtivery time frame. As such, we will run the model 12 more times, and each time increase the time frame around the introduction of Barge, from very small, to much larger.

In [27]:
# First, we make any empty list of models and names, to store models and names of models
models = []
names = []

# Then, we make a loop, in which we repeat this regression, but increase the range around 2010 by 1 year both ways each time.
for window in range(1,12):
    did = df[(df['year']>=(2010-window))&(df['year']<=(2010+window))&df['location'].isin(['Amapá','Pará'])]
    did['post']=np.where(did['year']>=2010,1,0)
    did['treatment']=np.where(did['location']=='Pará',1,0)
    did['post_treatment']=did['post']*did['treatment']
    did_model = ols('val_percent ~ post + treatment + post_treatment', did).fit()

    models.append(did_model)
    names.append('± '+str(window)+' Year')

# Finally, we will print this in a table.

table=summary_col(models,stars=True,model_names=names)
print(table)



                ± 1 Year  ± 2 Year  ± 3 Year  ± 4 Year  ± 5 Year  ± 6 Year  ± 7 Year  ± 8 Year  ± 9 Year
--------------------------------------------------------------------------------------------------------
Intercept      2.6246*** 2.5830*** 2.5418*** 2.5010*** 2.4607*** 2.4183*** 2.3727*** 2.3254*** 2.2789***
               (0.0771)  (0.0753)  (0.0828)  (0.0901)  (0.0956)  (0.1014)  (0.1070)  (0.1109)  (0.1127) 
post           0.1376    0.2403**  0.3485*** 0.4590*** 0.5681*** 0.6815*** 0.7975*** 0.9077*** 1.0081***
               (0.0944)  (0.0972)  (0.1095)  (0.1209)  (0.1294)  (0.1381)  (0.1466)  (0.1524)  (0.1554) 
treatment      0.7202**  0.7137*** 0.7066*** 0.6990*** 0.6909*** 0.6810*** 0.6679*** 0.6523*** 0.6354***
               (0.1090)  (0.1065)  (0.1171)  (0.1275)  (0.1352)  (0.1433)  (0.1514)  (0.1568)  (0.1594) 
post_treatment 0.0154    0.0263    0.0368    0.0468    0.0559    0.0613    0.0665    0.0764    0.0902   
               (0.1335)  (0.1374)  (0.1549)  (0.1710) 

Alas, here we see that, even when considering a wide range of years surrounding the implementation of Barge, there is still not sufficent evidence to reject our null hypothesis. Thus we can conclude that although there seems to be an increase in type 2 diabetes prevalence in Amapá after Barge's establishment in Pará, this could have been due to random chance, and so we cannot statistically confirm that there is a causal relationship here.